In [1]:
import os, sys
import numpy as np
import pandas as pd

In [2]:
proj_root = os.path.abspath("..")
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

Paths

In [3]:
RAW_DATA = os.path.join("..","data","raw")
PROCESSED_DATA = os.path.join("..","data","processed")

In [4]:
train_path = os.path.join(RAW_DATA, "train.csv")
test_path = os.path.join(RAW_DATA, "test.csv")

Load data

In [5]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [6]:
print("train shape:", df_train.shape)
print("test shape:", df_test.shape)

train shape: (1460, 81)
test shape: (1459, 80)


Check for missing values

In [7]:
missing = df_train.isnull().sum()
missing = missing[missing>0].sort_values(ascending = False)
missing_percent = (missing / len(df_train)) * 100
missing_summary = pd.DataFrame({"Missing values":missing, "Percent": missing_percent})
missing_summary

,Missing values,Percent
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
MasVnrType,872,59.726027
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageType,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945


Drop columns with >50% missing values

In [8]:
from src.preprocess import clean_data

df_train = pd.read_csv(os.path.join(RAW_DATA,"train.csv"))
df_test  = pd.read_csv(os.path.join(RAW_DATA,"test.csv"))

result = clean_data(df_train, df_test, drop_threshold=0.5)
print("Dropped:", result["dropped_columns"])
print("Missing after (train):", result["missing_after_train"])
print("Missing after (test):", result["missing_after_test"])
# if zero, save cleaned csvs
if result["missing_after_train"] == 0 and result["missing_after_test"] == 0:
    result["train"].to_csv("../data/processed/train_clean.csv", index=False)
    result["test"].to_csv("../data/processed/test_clean.csv", index=False)
    print("Saved processed files.")

df_train = result["train"]   # overwrites raw
df_test  = result["test"]

Dropped: ['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']
Missing after (train): 0
Missing after (test): 11


C:\Users\Mohammed\OneDrive\Desktop\ML Engineer\house-price-prediction\src\preprocess.py:101: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, col] = df[col].replace(QUALITY_MAP).fillna(0).astype(int)
C:\Users\Mohammed\OneDrive\Desktop\ML Engineer\house-price-prediction\src\preprocess.py:101: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, col] = df[col].replace(QUALITY_MAP).fillna(0).astype(int)
C:\Users\Mohammed\OneDrive\Desktop\ML Engineer\house-price-prediction\src\preprocess.py:101: FutureWarning: Downcasting be

Which columns are missing in test

In [9]:
missing_test = df_test.isnull().sum()
missing_test = missing_test[missing_test > 0].sort_values(ascending=False)
print("Missing columns in test (count):")
print(missing_test)


Missing columns in test (count):
MSZoning       4
Utilities      2
Functional     2
Exterior1st    1
Exterior2nd    1
SaleType       1
dtype: int64


Fill the specific categorical NaNs in test with train mode


In [10]:
fill_cols = ["MSZoning", "Utilities", "Functional", "Exterior1st", "Exterior2nd", "SaleType"]

for c in fill_cols:
    if c in df_test.columns:
        mode_val = df_train[c].mode(dropna=True)[0]
        df_test.loc[df_test[c].isnull(), c] = mode_val

# Verify again
print("Remaining missing in test after fix:")
print(df_test.isnull().sum()[df_test.isnull().sum() > 0])

Remaining missing in test after fix:
Series([], dtype: int64)


In [11]:
df_train.to_csv(os.path.join(PROCESSED_DATA, "train_clean.csv"), index=False)
df_test.to_csv(os.path.join(PROCESSED_DATA, "test_clean.csv"), index=False)

print("✅ Saved final cleaned files to data/processed/")

✅ Saved final cleaned files to data/processed/


# Outlier removal & basic feature engineering

In [12]:
train_path = os.path.join(PROCESSED_DATA, "train_clean.csv")
df = pd.read_csv(train_path)

print("Before outlier removal:", df.shape)

# Conservative rule: common Kaggle practice removes extremely large living-area outliers
outlier_mask = df["GrLivArea"] > 4000
print("Outliers to drop (GrLivArea > 4000):", outlier_mask.sum())




Before outlier removal: (1460, 78)
Outliers to drop (GrLivArea > 4000): 4


In [13]:
# Optionally inspect the rows:
display(df.loc[outlier_mask, ["Id","GrLivArea","SalePrice"]])



,Id,GrLivArea,SalePrice
523,524,4676,184750
691,692,4316,755000
1182,1183,4476,745000
1298,1299,5642,160000


In [14]:
# Drop those outliers
df = df.loc[~outlier_mask].reset_index(drop=True)
print("After outlier removal:", df.shape)



After outlier removal: (1456, 78)


In [15]:
# Basic engineered features (useful and safe):
if {"TotalBsmtSF","1stFlrSF","2ndFlrSF"}.issubset(df.columns):
    df["TotalSF"] = df["TotalBsmtSF"].fillna(0) + df["1stFlrSF"].fillna(0) + df["2ndFlrSF"].fillna(0)
if "YearBuilt" in df.columns and "YrSold" in df.columns:
    df["HouseAge"] = df["YrSold"] - df["YearBuilt"]

# Save final training file for modeling
df.to_csv(os.path.join(PROCESSED_DATA, "train_final.csv"), index=False)
print("Saved train_final.csv")

Saved train_final.csv
